In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df = df.filter(['icustay_id','icu_day','chloride_max'])
df = df.dropna(subset = ['chloride_max'])

# Chloride >= 110 in the second day
df_chl = df.query('icu_day == 2')
df_chl['chl_110'] = (df_chl['chloride_max'] >= 110).astype(int)

# Demographic
df = pd.read_csv('adm_demographics.csv')
df = df.set_index(['subject_id', 'hadm_id'])
df_demo = df.filter(['insurance','ethnicity','age','gender'])

# Weight
df = pd.read_csv('weight.csv')
df = df[df['icustay_id'].isin(icu)]
df_wt = df.filter(['icustay_id','day','weight'])

# GCS(Glasgow coma scale)
df = pd.read_csv('gcs_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df_gcs = df.filter(['icustay_id','day','mingcs'])

# First-day Vitals(Merge with GCS and Weight)
df = pd.read_csv('vitals_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df.filter(['day', 'heartrate_max','sysbp_min','diasbp_min','resprate_max','spo2_min','tempc_max'])
df_vitals = df_vitals.reset_index()
df_vitals = df_vitals.merge(df_wt, on = ['icustay_id','day']).merge(df_gcs, on = ['icustay_id','day'])
df_vitals = df_vitals.query('(day == 1) & (0 < heartrate_max <= 250) &(40<sysbp_min <190) &(20<diasbp_min<150) &(6<resprate_max<80) &(0<spo2_min<100)&(30<tempc_max<44)&(30<weight<500)&(3<mingcs<15)'
                           )
df_vitals = df_vitals.set_index(['subject_id','hadm_id','icustay_id'])

# Medication from prescription table
df = pd.read_csv('medication.csv')
df_med = df.drop(df.columns[[0,3]], axis=1)
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)
df = df_med.merge(df, on = ['subject_id','hadm_id'])
df['taken'] = ((pd.to_datetime(df['intime']) <= pd.to_datetime(df['startdate'])) & 
               ((pd.to_datetime(df['startdate']) <= (pd.to_datetime(df['intime']) + datetime.timedelta(days=1)))))
df = df.filter(['subject_id','hadm_id','icustay_id','drug','taken'])
df_med = df.set_index(['subject_id','hadm_id','icustay_id'])
df_med = df_med.groupby(['subject_id','hadm_id','icustay_id','drug']).any()
df_med = df_med['taken'].unstack()
df_med = (df_med * 1).fillna(0)

# Intake and output
df = pd.read_csv('fluid_io.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.query('icu_day == 1')
df_fluid = df.set_index(['subject_id','hadm_id','icustay_id'])
df_fluid.head()

# Diagnosis Code(group by elixhauser)
df = pd.read_csv('elixhauser.csv')
df_icd = df.drop(df.columns[[0]], axis=1)
df_icd = df_icd.set_index('hadm_id')

# Labs
df = pd.read_csv('labs1.csv')
df = df.drop(df.columns[0], axis=1)
df = df[df['icustay_id'].isin(icu)]
df_labs = df.query('day == 1 & 0.1<lactate_max<40 & 90<sodium_max<190 & 1.5<potassium_max<15 & 1<albumin_max<6.5 & 5<bicarbonate_min<65 & 0.1<creatinine_max<47 & 66<chloride_max<170 & 18<glucose_max<2500 & 2<hemoglobin_min<21 & 5<platelet_min<1600  & 0.5<inr_max<20 & 1<bun_max<280 & 0.1<wbc_max<680 & 0.1<bilirubin_min<80 & 1.5<calcium_total_min<18 & 0.2<calcium_ionized_min<2.5 & 2<ast_sgot_max<25000 & 3<amylase_max<25000 &  1<lipase_max<25000 & 0.1<c_reactive_protein_max<300')
df_labs = df_labs.set_index(['subject_id','hadm_id','icustay_id'])

# Interventions
df = pd.read_csv('interventions.csv')
df = df.query('day == 1')
df_int = df.drop(df.columns[[0]], axis=1)
df_int = df_int.set_index(['subject_id', 'hadm_id', 'icustay_id'])
df_int = df_int.filter(['label'])
df_int['taken'] = 1
df_int = df_int.groupby(['subject_id','hadm_id','icustay_id','label']).any()
df_int = df_int['taken'].unstack()
df_int = (df_int * 1).fillna(0)

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
demo = ['age', 'gender', 'ethnicity']
vitals = ['heartrate_max', 'sysbp_min', 'diasbp_min', 'resprate_max', 'weight', 'mingcs']
medication = ['norepinephrine']
fluid = ['fluid_net_input_ml']
diag = ['congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 
        'chronic_pulmonary', 
       'diabetes_complicated',  'renal_failure',
       'liver_disease',  
        'coagulopathy',  'fluid_electrolyte',
       'blood_loss_anemia']
labs = ['chloride_max', 'sodium_max', 'bicarbonate_min']
interventions = ['EPAP','IPAP','LPM','MeanAirwayPressure']

In [4]:
df_demo = df_demo[demo]
df_vitals = df_vitals[vitals]
df_med = df_med[medication]
df_fluid = df_fluid[fluid]
df_icd = df_icd[diag]
df_labs = df_labs[labs]
df_int = df_int[interventions]
df_chl = df_chl['chl_110']

In [8]:
df = df_demo.join(df_chl, how = 'inner') \
.join(df_fluid, how = 'left') \
.join(df_icd, how = 'left') \
.join(df_labs, how = 'left') \
.join(df_int, how='left') \
.join(df_vitals, how='left') \
.join(df_med, how = 'left') 
df

age gender ethnicity  chl_110  \
subject_id hadm_id icustay_id                                        
3          145834  211552      76.575342      M     WHITE        1   
4          185777  294638      47.876712      F     WHITE        0   
6          107064  228232      65.983562      F     WHITE        0   
9          150750  220597      41.816438      M     OTHER        0   
12         112213  232669      72.419178      M     WHITE        1   
13         143045  263738      39.890411      F     WHITE        1   
17         194023  277042      47.484932      F     WHITE        0   
21         109451  217847      87.495890      M     WHITE        0   
           111970  216859      87.882192      M     WHITE        0   
26         197661  244882      72.052055      M     OTHER        0   
30         104557  225176      91.400000      M     OTHER        0   
31         128652  254478      72.312329      M     WHITE        0   
32         175413  295037      45.542466      M     WHITE        0   
33         176176  296681      82.446575      M     OTHER        0   
35         166707  282039      69.487671      F     WHITE        0   
36         122659  211200      69.778082      M     WHITE        0   
           165660  241249      72.775342      M     WHITE        0   
38         185910  248910      75.991781      M     WHITE        1   
41         101757  261027      56.671233      M     WHITE        0   
42         119203  210828      61.205479      M     OTHER        0   
43         146828  225852      32.789041      M     WHITE        0   
44         181750  291554      80.235616      M     OTHER        0   
45         157907  274249      42.432877      M     WHITE        0   
52         190797  261857      38.147945      M     WHITE        0   
53         155385  239839      45.580822      M     OTHER        0   
55         190665  262723      64.202740      F     WHITE        0   
56         181711  275642      91.400000      F     WHITE        0   
59         104130  224440      87.150685      F     WHITE        0   
61         176332  252348      54.690411      M     WHITE        1   
           189535  217135      55.241096      M     WHITE        1   
...                                  ...    ...       ...      ...   
99863      100749  216757      42.720548      M     WHITE        0   
99865      189346  284772      77.515068      F     WHITE        0   
99868      177777  231346      20.452055      F     OTHER        0   
99873      143544  206270      45.643836      M     WHITE        1   
99880      179507  215004      87.961644      F     OTHER        0   
99881      172327  214377      77.816438      M     WHITE        0   
99883      150755  276467      73.931507      M     WHITE        0   
99893      128349  208873      88.608219      M     WHITE        0   
99894      127380  204236      60.041096      F     WHITE        1   
99897      162913  266801      53.586301      M     BLACK        0   
99899      188409  245756      87.013699      M     BLACK        0   
99901      131711  226541      68.873973      M     WHITE        0   
99912      189380  236183      84.600000      M     WHITE        1   
99913      175989  292375      71.136986      M  HISPANIC        0   
99923      192053  298857      54.646575      M     WHITE        0   
99936      107913  213906      91.400000      F     WHITE        0   
99938      103815  296707      80.890411      M     WHITE        0   
99939      159023  252442      26.967123      M  HISPANIC        0   
99944      185654  221067      85.536986      F     WHITE        0   
99955      108494  243255      63.232877      M     BLACK        1   
99957      148574  257127      68.095890      M     WHITE        0   
99966      167228  252173      77.536986      M     WHITE        0   
99973      150202  275083      65.542466      F     WHITE        1   
99982      112748  275958      65.304110      M     WHITE        0   
           151454  221194      65.200000      M

In [9]:
df.to_csv('dataFirst.csv')